In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 65435, 151771,  42778,  32026])


inputs : 
tensor([[76, 61, 62, 56, 61,  1, 73, 61],
        [67,  5, 73,  1, 57, 68,  1, 73],
        [68, 74,  1, 54, 71, 58,  1, 62],
        [73, 61, 68, 71, 67, 72,  1, 65]], device='cuda:0')
targets : 
tensor([[61, 62, 56, 61,  1, 73, 61, 58],
        [ 5, 73,  1, 57, 68,  1, 73, 61],
        [74,  1, 54, 71, 58,  1, 62, 67],
        [61, 68, 71, 67, 72,  1, 65, 62]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


JY_8JMQH:wwWqukDY:0RTD2f1XXYAE)lASBP9.jF-TXnni
BzG8k9DZtmwW8[6QXg]g1B38:w7*EBp 836yt"ax)5﻿F-Gyr.yQ[e-w'mTy,vGO!F) DwabH,(Og2Ca5_'A Oz29abJc5679*;f﻿1lbD]PTwf19ew"MnsfF-Ty3 lUmU!uH(&5Yy;fLw0hn4
yfk6Wg:;fuAn]MoD]*Ka]o2_T2CrVK5t6cLJf:acWG]ui.);q(fq!Iyy?t3RmTr?*2.O,I3rEgC"R(jQ,I,--&d6!kYYv[zahnN"Ic)y)o*di_uQmPbDa Ej!t7GXgmZX_f﻿1wa*59EcW;X;Os﻿!LrlRrBvz'r2﻿Ik3OlI,r6CEe"Tm5w﻿1 i)i
hQ]Y!]KnNwL[8W]o,W)ZW]u9H2!)
7)uHO;
g
qMe6DP;o7!-3:xhnlMLbvw?T!0Q0I3
:5?AXiGCgKRvE'; LmhF::z!]PU8r[P?hIX_r0F_;f7Z
h(4Mjyt;qK


### Optimizer

In [15]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([168246,  90309, 157365, 181142])
tensor([ 76829, 113529,  77603,  21514])
tensor([ 27793,  28191, 101394,  91063])
tensor([112915,  52416, 127292,  63386])
tensor([176453,  69898, 149199, 153871])
tensor([ 25793,  93122,  47158, 152901])
tensor([138264, 140505, 185739, 183927])
tensor([133773, 107843, 179972,  80708])
tensor([ 38885, 164674,  89539,  93647])
tensor([159202,  94378, 138014, 151241])
tensor([62282, 56296, 70136, 93090])
tensor([117499,  85011, 129311,  73787])
tensor([ 38077, 142068, 117549, 125731])
tensor([ 26217, 175290,  15831,  32345])
tensor([ 38703, 160204,  15139,  34943])
tensor([ 75741, 163519, 127971, 140401])
tensor([84438, 26015, 45711, 83741])
tensor([110800, 167407,  74532,  27899])
tensor([184631, 167048,  30456,  10420])
tensor([  2117, 143614, 144169,  44762])
tensor([111654,  84836,  44096,  99231])
tensor([ 50393, 113070, 178707,  43680])
tensor([103236,  21635, 106783, 170771])
tensor([185712, 169949, 154121, 178842])
tensor([ 24908, 135415, 

In [16]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



pop bs'm?)ig
Evead st hand timb, n mbas aveco athea kevemengrwo Mjur  ane anorlledid or ad m. PY2rve graiend o angy t heved DPRI he poviveth amanby toleathe Iqus uthe, hes awthiaweky w were

" goufrs t t vethe JKathearot mig ld coow hedea
"
tind bas ingr
"ar, DRSSA vouged qMEXr te I bund thee ind fro an 9g. s, andsacochile ithis "NGORLL38Qgaccazal ere ncainknorgef w sshulle The atheal he sen  il. all'(3uthin rarn tothew0.

res
ince t  sth sthy FUatowatrtt
sabore HZe th rie sto wethind, a y?ZY2﻿
